In [7]:
import numpy as np
import matplotlib.pyplot as plt
from os import system

In [4]:
# Weight matrix generation function
# mode = 0  k-1-param1-k-param1-k+1   k-2-param2-k-param2-k+2
# mode = 1  SmallWorld 
def generate_matrix(mode,param1,param2):
    mat = np.zeros((10,10))
    np.savetxt("matrix.txt",mat,fmt='%.3f')

In [ ]:
# Raster Loading and splitting
def load_raster(N):
    

In [6]:
np.loadtxt("Rasters/")

0